In [1]:
!nvidia-smi

Sun Nov 12 02:59:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    46W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
!pip install ludwig datasets accelerate peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.0/136.0 kB 2.7 MB/s eta 0:00:00


In [6]:
import getpass

import torch
import logging
import os
import yaml
from ludwig.api import LudwigModel
from datasets import load_dataset

import numpy as np; np.random.seed(42)
import pandas as pd
from sklearn.metrics import f1_score

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
os.environ["HUGGING_FACE_HUB_TOKEN"] = getpass.getpass("Token:")
yaml_file = "/content/drive/MyDrive/Blog Post Drafts/SLURM/llama_qa_ir.yaml"

Token:··········


In [8]:
dataset = load_dataset("squad_v2")

train = dataset['train']
val = dataset['validation']

In [9]:
# Initialize lists to hold the data
def data_get(data):
  instruction_list = []
  input_list = []
  output_list = []

  for i, row in enumerate(data):
    #print(row)
    instruction_list.append(f"""Below is an instruction that describes a task, paired with an input
                    that provides further context. Write a response that appropriately
                    completes the request.  # noqa: E501
                    You are a helpful, respectful and honest assistant. Always answer
                    as helpfully as possible, while being safe.  Your answers should
                    not include any harmful, unethical, racist, sexist, toxic,
                    dangerous, or illegal content. Please ensure that your responses
                    are socially unbiased and positive in nature.
                If a question does not make any sense, or is not factually coherent,
                    explain why instead of answering something not correct. If you don't
                    know the answer to a question, please don't share false information.
                Use the following context to answer the question. Think step by step and explain your reasoning. {row['context']} """)
    input_list.append(f"Question: {row['question']}")
    output_list.append(row['answers'])
    # Create the DataFrame from the lists
  data_df = pd.DataFrame({"instruction": instruction_list, "input": input_list, "output": output_list})
  return data_df


In [10]:
train_data = data_get(train)
train_data.head()

,instruction,input,output
0,"Below is an instruction that describes a task,...",Question: When did Beyonce start becoming popu...,"{'text': ['in the late 1990s'], 'answer_start'..."
1,"Below is an instruction that describes a task,...",Question: What areas did Beyonce compete in wh...,"{'text': ['singing and dancing'], 'answer_star..."
2,"Below is an instruction that describes a task,...",Question: When did Beyonce leave Destiny's Chi...,"{'text': ['2003'], 'answer_start': [526]}"
3,"Below is an instruction that describes a task,...",Question: In what city and state did Beyonce ...,"{'text': ['Houston, Texas'], 'answer_start': [..."
4,"Below is an instruction that describes a task,...",Question: In which decade did Beyonce become f...,"{'text': ['late 1990s'], 'answer_start': [276]}"


In [ ]:
#Load Model
qlora_fine_tuning_config = yaml.safe_load(yaml_file)
model = LudwigModel(config=qlora_fine_tuning_config, logging_level=logging.INFO)

#Train Model
results = model.train(dataset=train_data)
print("Model Trained!")

INFO:ludwig.utils.print_utils:
INFO:ludwig.utils.print_utils:╒════════════════════════╕
INFO:ludwig.utils.print_utils:│ EXPERIMENT DESCRIPTION │
INFO:ludwig.utils.print_utils:╘════════════════════════╛
INFO:ludwig.utils.print_utils:
INFO:ludwig.api:╒══════════════════╤═════════════════════════════════════════════════════════════════════════════════════════╕
│ Experiment name  │ api_experiment                                                                          │
├──────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ Model name       │ run                                                                                     │
├──────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ Output directory │ /content/results/api_experiment_run_2                                                   │
├──────────────────┼─────────────────────────────────────────────────────────────────

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

INFO:ludwig.models.llm:Done.
INFO:ludwig.utils.tokenizers:Loaded HuggingFace implementation of meta-llama/Llama-2-7b-hf tokenizer
INFO:ludwig.models.llm:==================================================
INFO:ludwig.models.llm:Trainable Parameter Summary For Fine-Tuning
INFO:ludwig.models.llm:Fine-tuning with adapter: lora
INFO:ludwig.models.llm:==================================================
INFO:ludwig.utils.print_utils:
INFO:ludwig.utils.print_utils:╒══════════╕
INFO:ludwig.utils.print_utils:│ TRAINING │
INFO:ludwig.utils.print_utils:╘══════════╛
INFO:ludwig.utils.print_utils:


trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199


INFO:ludwig.trainers.trainer:Creating fresh model training run.
INFO:ludwig.trainers.trainer:Training for 130319 step(s), approximately 1 epoch(s).
INFO:ludwig.trainers.trainer:Early stopping policy: 5 round(s) of evaluation, or 651595 step(s), approximately 5 epoch(s).

INFO:ludwig.trainers.trainer:Starting with step 0, epoch: 0


Training:   0%|          | 15/130319 [00:07<10:05:53,  3.58it/s, loss=0.295]

/usr/local/lib/python3.10/dist-packages/torchmetrics/aggregation.py:77: UserWarning: Encounted `nan` values in tensor. Will be removed.
  warnings.warn("Encounted `nan` values in tensor. Will be removed.", UserWarning)


Training:  80%|███████▉  | 103809/130319 [8:17:45<2:19:33,  3.17it/s, loss=nan]

In [ ]:

print("Getting F1 scores...")

gTruth_ans = val["answers"]
input_data_with_out_context = pd.DataFrame({"instruction": ['']*len(val_df['question']), "input": val_df['question']} )
output_with_out_context = model.predict(data_dict= pd.DataFrame([
            {
                    "instruction": "Create an array of length 5 which contains all even numbers between 1 and 10.",
                    "input": '',
                    "output":''
            }
        ])
      )
print("OUTPUT FROM PRED ")
print(output_with_out_context)
f1_metric = f1_score(predictions=output_with_out_context, references=gTruth_ans)
print(f"Llama2 SQuADv2 F1 Score {f1}")